In [5]:
from semantic_kernel.kernel import Kernel
import asyncio
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.planners import SequentialPlanner
from dotenv import load_dotenv
import os
from semantic_kernel.planners import SequentialPlanner
from semantic_kernel.contents.chat_history import ChatHistory  # noqa: F401
from semantic_kernel.functions.kernel_arguments import KernelArguments  # noqa: F401
from semantic_kernel.prompt_template.input_variable import InputVariable  # noqa: F401
# Get ROOT_DIR
ROOT_DIR = os.chdir(os.path.dirname(os.path.abspath('__file__')))

# Take environment variables from .env.
#load_dotenv(ROOT_DIR+'/.env', override=True)


load_dotenv(verbose=True)
async def create_plan(planner, input):
    return await planner.create_plan(goal=input)

async def invoke_plan(sequential_plan):
    return await sequential_plan.invoke()

async def main(nlp_input):
    #from semantic_kernel.services import Service
    #from samples.service_settings import ServiceSettings
    #
    #service_settings = ServiceSettings.create()
    #
    ## Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
    #selectedService = (
    #    Service.AzureOpenAI
    #    if service_settings.global_llm_service is None
    #    else Service(service_settings.global_llm_service.lower())
    #)
    #print(f"Using service type: {selectedService}")
    #kernel = Kernel()
    #service_id = None
    #if selectedService == Service.OpenAI:
    #    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
    #
    #    service_id = "default"
    #    kernel.add_service(
    #        OpenAIChatCompletion(
    #            service_id=service_id,
    #        ),
    #    )
    #elif selectedService == Service.AzureOpenAI:
    #    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
    #
    #    service_id = "default"
    #    kernel.add_service(
    #        AzureChatCompletion(
    #            service_id=service_id,
    #        ),
    #    )
    #kernel = sk.Kernel()
    #service_id = "default"
    #kernel.add_service(AzureChatCompletion(
    #    service_id=service_id,
    #    deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    #    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    #    base_url=os.getenv("AZURE_OPENAI_ENDPOINT")
    #))
    # Get AOAI settings from .env
    #deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()

    # Set the deployment name to the value of your chat model
    #deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
    #api_key = os.getenv("AZURE_OPENAI_API_KEY")
    #endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    #azure_text_service = AzureChatCompletion(deployment_name=deployment, endpoint=endpoint, api_key=api_key)
    kernel = Kernel()
    service_id = "default"
    kernel.add_service(
        AzureChatCompletion(
            service_id=service_id,
        ),
    )    

    from plugins.QueryDb import queryDb as plugin
    QueryDbPlugin = plugin.QueryDbPlugin(os.getenv("CONNECTION_STRING"))
    kernel.add_plugin(QueryDbPlugin,plugin_name="QueryDbPlugin")
    # Immport NLP to SQL Plugin
    plugins_directory = "./plugins"
    kernel.add_plugin(parent_directory=plugins_directory, plugin_name="nlpToSqlPlugin")
    #kernel.import_plugin(plugin.QueryDbPlugin(os.getenv("CONNECTION_STRING")), plugin_name="QueryDbPlugin")
    #kernel.import_semantic_plugin_from_directory(plugins_directory, "nlpToSqlPlugin")
    #kernel.import_plugin(plugin.QueryDbPlugin(os.getenv("CONNECTION_STRING")), plugin_name="QueryDbPlugin")
    #summarize_plugin = kernel.add_plugin(plugin_name="QueryDbPlugin", parent_directory=plugins_directory)
    for plugin_name, plugin in kernel.plugins.items():
        for function_name, function in plugin.functions.items():
            print(f"Plugin: {plugin_name}, Function: {function_name}")
    
    planner = SequentialPlanner(kernel, service_id)

    # Create a plan with the NLP input
    ask = f"「{nlp_input}」の要求に対して、SQLクエリーを作成し、クエリーを使ってデータベース問い合わせて、必ずデータベースから得た情報を答えること。"
    #from semantic_kernel.planners.function_calling_stepwise_planner import (
    #    FunctionCallingStepwisePlanner,
    #    FunctionCallingStepwisePlannerOptions,
    #)
    #
    #questions = [
    #     f"「{nlp_input}」の要求に対して、SQLクエリーを作成し、クエリーを使ってデータベース問い合わせて、必ずデータベースから得た情報を答えること。"
    #]
    #
    #options = FunctionCallingStepwisePlannerOptions(
    #    max_iterations=10,
    #    max_tokens=4000,
    #)

    #planner = FunctionCallingStepwisePlanner(service_id=service_id, options=options)
    #for question in questions:
    #    result = await planner.invoke(kernel, question)
    #    print(f"Q: {question}\nA: {result.final_answer}\n")
    plan = await create_plan(planner, ask)
    for index, step in enumerate(plan._steps):
        print("Step:", index)
        print(
            f"- {step.description.replace('.', '') if step.description else 'No description'} using {step.metadata.fully_qualified_name} with parameters: {step.parameters}"
        )
    ## Invoke the plan and get the result
    result = await plan.invoke(kernel)

    print('/n')
    print(f'User ASK: {nlp_input}')
    print(f'Response: {result}')
    print('/n')
    import json
    # Print each step of the plan and its result
    #for index, step in enumerate(plan._steps):
    #    print("Step:", index)
    #    print("Description:", step.description)
    #    print("Function:", step.plugin_name + "." + step._function.name)
    #    
    #    if len(step._outputs) > 0:
    #        #print("  Output:\n", str.replace(result[step._outputs[0]], "\n", "\n  "))
    #        print(step._outputs[0])
    #        print("\n")

In [6]:

import asyncio

#asyncio.run(main("売上一位の会社を教えてください。"))
await main("売上1位の会社名売上の総額を教えてください")
#asyncio.run(main("Give me the name of the best seller in terms of sales volume in the whole period"))
#asyncio.run(main("Which product has the highest sales volume in the last month"))

Plugin: QueryDbPlugin, Function: query_db
Plugin: nlpToSqlPlugin, Function: ConvertNLPToSQL
Plugin: nlpToSqlPlugin, Function: MakeSQLCompatible
Plugin: nlpToSqlPlugin, Function: WriteResponse
Step: 0
- 自然言語で記述されたSQLクエリを書く using nlpToSqlPlugin-ConvertNLPToSQL with parameters: {'input': '売上1位の会社名売上の総額を教えてください'}
Step: 1
- ANSI SQL方言のSQLクエリをTransact-SQL方言に変換します。 using nlpToSqlPlugin-MakeSQLCompatible with parameters: {'input': '$SQL_QUERY'}
Step: 2
- SQLクエリを使用してデータベースに問い合わせる using QueryDbPlugin-query_db with parameters: {'input': '$TSQL_QUERY'}
Step: 3
- データベースのクエリ結果に対する親切な応答を書く using nlpToSqlPlugin-WriteResponse with parameters: {'input': '$DB_RESULT'}
input=☆T-SQL:
```sql
WITH 売上集計 AS (
    SELECT 
        得意先コード,
        SUM(単価 * 数量 * (1 - 割引)) AS 総売上
    FROM 
        data.受注明細 AS 明細
    JOIN 
        data.受注 AS 受注 ON 明細.受注コード = 受注.受注コード
    GROUP BY 
        得意先コード
)
SELECT 
    得意先.得意先名,
    売上集計.総売上
FROM 
    売上集計
JOIN 
    data.得意先 ON 売上集計.得意先コード = 得意先.得意先コード
ORDER BY 
    売上集計.総売上